In [37]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/IPython/core/magics/pylab.py:162: UserWarning: pylab import has clobbered these variables: ['copy']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [38]:
import math
import os
import copy
import time

from tempfile import TemporaryDirectory
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

from sklearn.cluster import KMeans

In [101]:
songString = "CCGGAAGFFEEDDCGGFFEEDGGFFEEDCCGGAAGFFEEDDC"
notes=list(unique(list(songString)))
chord=[[0],
       [1],
       [2],
       [3],
       [4],
       [5]]
DaisyBell=list()
DaisyBellMatrix=[]
for note in list(songString):
    row=[0]*6
    row[notes.index(note)]=1
    DaisyBellMatrix.append(row)
    DaisyBell.append(chord[notes.index(note)])
print(DaisyBell)
DaisyBellMatrix=numpy.array(DaisyBellMatrix)

[[0], [5], [4], [2], [3], [2], [2], [1], [5], [1], [4], [3], [3], [4], [0], [0], [3], [4], [4], [5], [5], [2], [5], [3], [3], [1], [3], [1], [2], [1], [1], [5], [1], [3], [3], [0], [0], [3], [2], [3], [2], [0]]
42


In [54]:
HotDogsongString = "AACCGGGFFFFDDGCCFFDDDAAFFCCDCCGGGGGFFEEDDC"
notes=list(unique(list(songString)))
chord=[[0],
       [1],
       [2],
       [3],
       [4],
       [5]]
HotDog=list()
HotDogMatrix=[]
for note in list(HotDogsongString):
    row=[0]*6
    row[notes.index(note)]=1
    HotDogMatrix.append(row)
    HotDog.append(chord[notes.index(note)])
print(HotDog)
HotDogMatrix=numpy.array(HotDogMatrix)

[[0], [0], [1], [1], [5], [5], [5], [4], [4], [4], [4], [2], [2], [5], [1], [1], [4], [4], [2], [2], [2], [0], [0], [4], [4], [1], [1], [2], [1], [1], [5], [5], [5], [5], [5], [4], [4], [3], [3], [2], [2], [1]]


In [109]:
songStrings = numpy.array([
    "CFACHFDFEEDHEGBFHACGAHAEBHCBBFGDHGDBBCABHC",
    "CCGGAAGFFEEDDCGGFFEEDGGFFEEDCCGGAAGFFEEDDC",
    "ABCDEFABCDEFABCDEFABCDEFABCDEFABCDEFABCDEF",
    "ABACADAEAFABEFADECBABCFEDEFABCADEBACADFABE",
    "DBCACBCFFDCEFFEFCDDEFEBEACFECBBBCBECBFDAFB",
    "ABEBCAEFCDFFBCBDBBBCEDCBFBFFECBCEBCAAFFADB",
    "BEEFBAFDAEAAEFDBDFDEFCACEBCCDACEACACEEDBAA",
    "BFEBFEEBDBCFEAACAAAFDFCBFBFEAACFFCAABCEDDC",
    "BADDFFEADBEDFDFBEBCCADEFDEABBFDEFFEBEEFDEF",
    "ABFFEDBDBFECEDEAEBBEECFDDAEDCDBBFCADADBBCF",
    "DFBCEBDAADAAFCDACADDAFFACDCFCCDDDCFBEBBDED",
    "CCFBEFDDCBFDADDBFBCCEEABAFAAAEDCDCEAEFBFCD",
    "EBADFFAAFADDDABEABBDFDCAFBCDEEBBBECDDFEEAE",
    "AFADDFEFADDBCDCFEEFCAEEEDFFEDBCADBBDBAEFCD"])
notes=list("ABCDEFGH")
print(notes)
chord=[[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7]]
allSongs=list()
for songString in songStrings:
    song=[]
    for note in list(songString):
        row=[0]*8
        row[notes.index(note)]=1
        song.append(chord[notes.index(note)])
    allSongs.append(song)
for song in allSongs:
    print(song)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
[[2], [5], [0], [2], [7], [5], [3], [5], [4], [4], [3], [7], [4], [6], [1], [5], [7], [0], [2], [6], [0], [7], [0], [4], [1], [7], [2], [1], [1], [5], [6], [3], [7], [6], [3], [1], [1], [2], [0], [1], [7], [2]]
[[2], [2], [6], [6], [0], [0], [6], [5], [5], [4], [4], [3], [3], [2], [6], [6], [5], [5], [4], [4], [3], [6], [6], [5], [5], [4], [4], [3], [2], [2], [6], [6], [0], [0], [6], [5], [5], [4], [4], [3], [3], [2]]
[[0], [1], [2], [3], [4], [5], [0], [1], [2], [3], [4], [5], [0], [1], [2], [3], [4], [5], [0], [1], [2], [3], [4], [5], [0], [1], [2], [3], [4], [5], [0], [1], [2], [3], [4], [5], [0], [1], [2], [3], [4], [5]]
[[0], [1], [0], [2], [0], [3], [0], [4], [0], [5], [0], [1], [4], [5], [0], [3], [4], [2], [1], [0], [1], [2], [5], [4], [3], [4], [5], [0], [1], [2], [0], [3], [4], [1], [0], [2], [0], [3], [5], [0], [1], [4]]
[[3], [1], [2], [0], [2], [1], [2], [5], [5], [3], [2], [4], [5], [5], [4], [5], [2], [3], [3], [4], [5], [4], [1],

In [41]:
def get_batch(source, i, song_index, bptt=42):
    data=[]
    target=[]
    for pos in range(i,i+bptt):
        data.append(source[song_index][pos%len(source[song_index])])
        target+=source[song_index][(pos+1)%len(source[song_index])]
    return data, target

In [42]:
class TransformerModel(nn.Module):

    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor, verbose=False) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = self.encoder(src) * math.sqrt(self.d_model)
        if verbose:
            print(src.shape)
            figure()
            imshow(src.detach().numpy().reshape((42,20)))
        src = self.pos_encoder(src)
        if verbose:
            print(src.shape)
            figure()
            imshow(src.detach().numpy().reshape((42,20)))
        output = self.transformer_encoder(src, src_mask)
        if verbose:
            print(output.shape)
            figure()
            imshow(output.detach().numpy().reshape((42,20)))
        output = self.decoder(output)
        if verbose:
            print(output.shape)
            figure()
            imshow(output.detach().numpy().reshape((42,6)))
        return output


def generate_square_subsequent_mask(sz: int) -> Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

In [53]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [44]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [45]:
model = TransformerModel(ntoken=8, d_model=64, nhead=4, d_hid=256, nlayers=2, dropout=0.0)
print(model)

TransformerModel(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=256, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
        (linear2): Linear(in_features=256, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.2, inplace=False)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
    )
  )
  (encoder): Embedding(6, 64)
  (decoder): Linear(in_features=64, out_features=6, bias=True)
)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [46]:
model.state_dict()

OrderedDict([('pos_encoder.pe',
              tensor([[[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
                         0.0000e+00,  1.0000e+00]],
              
                      [[ 8.4147e-01,  5.4030e-01,  6.8156e-01,  ...,  1.0000e+00,
                         1.3335e-04,  1.0000e+00]],
              
                      [[ 9.0930e-01, -4.1615e-01,  9.9748e-01,  ...,  1.0000e+00,
                         2.6670e-04,  1.0000e+00]],
              
                      ...,
              
                      [[ 9.5625e-01, -2.9254e-01,  6.4315e-01,  ...,  6.3049e-01,
                         6.1813e-01,  7.8608e-01]],
              
                      [[ 2.7050e-01, -9.6272e-01, -5.1133e-02,  ...,  6.3036e-01,
                         6.1823e-01,  7.8599e-01]],
              
                      [[-6.6395e-01, -7.4778e-01, -7.1816e-01,  ...,  6.3022e-01,
                         6.1834e-01,  7.8591e-01]]])),
             ('transformer_encoder.layers.0.

In [80]:

def train(model, songs, optimizer, criterion, bptt=42, device='cpu'):
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(bptt).to(device)

    batches = numpy.array(list(range(bptt)))
    numpy.random.shuffle(batches)
    for song_index, song in enumerate(songs):
        num_batches = len(song) // bptt
        for batch, i in enumerate(batches):
            #print(f"batch {batch} of {num_batches}")
            #print(f"i {i}")
            data, targets = get_batch(songs, i, song_index, bptt)
            seq_len = len(data)
            if seq_len != bptt:  # only on last batch
                src_mask = src_mask[:seq_len, :seq_len]
            output = model(torch.tensor(data,dtype=torch.long), src_mask)
            loss = criterion(output.view(-1, ntokens), torch.tensor(targets,dtype=torch.long))

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            optimizer.step()

            total_loss += loss.item()
            if batch % log_interval == 0 and batch > 0:
                lr = scheduler.get_last_lr()[0]
                ms_per_batch = (time.time() - start_time) * 1000 / log_interval
                cur_loss = total_loss / log_interval
                ppl = math.exp(cur_loss)
                print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                    f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                    f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
                total_loss = 0
                start_time = time.time()

def evaluate(model, songs, criterion, bptt=42, device='cpu'):    
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for song_index, song in enumerate(songs):
            for i in range(0, len(song) - 1, bptt):
                data, targets = get_batch(songs, i, song_index, bptt)
                seq_len = len(data)
                if seq_len != bptt:
                    src_mask = src_mask[:seq_len, :seq_len]
                output = model(torch.tensor(data,dtype=torch.long), src_mask)
                output_flat = output.view(-1, ntokens)
                total_loss += seq_len * criterion(output_flat, torch.tensor(targets,dtype=torch.long)).item()
    avg_loss = total_loss / sum(len(song) for song in songs)
    return avg_loss

In [165]:
#songs = [DaisyBell, hotDog]
#songs = [DaisyBell]
songs = allSongs[:5]
print(songs)

[[[2], [5], [0], [2], [7], [5], [3], [5], [4], [4], [3], [7], [4], [6], [1], [5], [7], [0], [2], [6], [0], [7], [0], [4], [1], [7], [2], [1], [1], [5], [6], [3], [7], [6], [3], [1], [1], [2], [0], [1], [7], [2]], [[2], [2], [6], [6], [0], [0], [6], [5], [5], [4], [4], [3], [3], [2], [6], [6], [5], [5], [4], [4], [3], [6], [6], [5], [5], [4], [4], [3], [2], [2], [6], [6], [0], [0], [6], [5], [5], [4], [4], [3], [3], [2]], [[0], [1], [2], [3], [4], [5], [0], [1], [2], [3], [4], [5], [0], [1], [2], [3], [4], [5], [0], [1], [2], [3], [4], [5], [0], [1], [2], [3], [4], [5], [0], [1], [2], [3], [4], [5], [0], [1], [2], [3], [4], [5]], [[0], [1], [0], [2], [0], [3], [0], [4], [0], [5], [0], [1], [4], [5], [0], [3], [4], [2], [1], [0], [1], [2], [5], [4], [3], [4], [5], [0], [1], [2], [0], [3], [4], [1], [0], [2], [0], [3], [5], [0], [1], [4]], [[3], [1], [2], [0], [2], [1], [2], [5], [5], [3], [2], [4], [5], [5], [4], [5], [2], [3], [3], [4], [5], [4], [1], [4], [0], [2], [5], [4], [2], [1], 

In [166]:
len(songs)

5

In [167]:
for song in songs:
    print(len(song))

42
42
42
42
42


In [ ]:
#ntokens = 6  # size of vocabulary
ntokens = 8  # size of vocabulary
emsize = 120  # embedding dimension
d_hid = 120  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 4  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 8  # number of heads in nn.MultiheadAttention
dropout = 0.0  # dropout probability

In [174]:
bptt = 42

#ntokens = 6  # size of vocabulary
ntokens = 8  # size of vocabulary
emsize = 64  # embedding dimension
d_hid = 256  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 4  # number of heads in nn.MultiheadAttention
dropout = 0.0  # dropout probability

criterion = nn.CrossEntropyLoss()
lr = 5.0  # learning rate
model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)


epochs=200

for epoch in range(1, epochs + 1):
    train(model, songs, optimizer, criterion)
    # Evaluate on the validation dataset
    val_loss = evaluate(model, songs, criterion)  # Replace `songs` with your validation dataset if available
    
    # Calculate and print the validation perplexity
    val_ppl = math.exp(val_loss)
    print(f'| End of epoch {epoch:3d} | Validation Loss: {val_loss:.2f} | Validation Perplexity: {val_ppl:8.2f}')

    scheduler.step()


/var/folders/87/pnb9p7_n30s29gprj1gbqbbm0000gn/T/ipykernel_29870/4199152564.py:53: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  avg_loss = total_loss / sum(len(song) for song in songs)


| End of epoch   1 | Validation Loss: 2.61 | Validation Perplexity:    13.56
| End of epoch   2 | Validation Loss: 2.69 | Validation Perplexity:    14.73
| End of epoch   3 | Validation Loss: 2.38 | Validation Perplexity:    10.77
| End of epoch   4 | Validation Loss: 2.58 | Validation Perplexity:    13.19
| End of epoch   5 | Validation Loss: 2.57 | Validation Perplexity:    13.12
| End of epoch   6 | Validation Loss: 2.66 | Validation Perplexity:    14.23
| End of epoch   7 | Validation Loss: 2.52 | Validation Perplexity:    12.37
| End of epoch   8 | Validation Loss: 2.61 | Validation Perplexity:    13.60
| End of epoch   9 | Validation Loss: 2.86 | Validation Perplexity:    17.45
| End of epoch  10 | Validation Loss: 2.65 | Validation Perplexity:    14.12
| End of epoch  11 | Validation Loss: 2.77 | Validation Perplexity:    15.90
| End of epoch  12 | Validation Loss: 3.01 | Validation Perplexity:    20.34
| End of epoch  13 | Validation Loss: 2.77 | Validation Perplexity:    16.01

In [175]:
for i ,song in enumerate(songs):
    for j in range(bptt):
        data,target=get_batch(songs,j, i)
        # Convert the input notes to a tensor and add an extra dimension

        # Create a mask for the input
        src_mask = generate_square_subsequent_mask(len(data)).to(device)

        # Run the model on the input tensor
        output = model(torch.tensor(data,dtype=torch.long),src_mask)

        # Detach the output from the computation graph and convert to numpy array
        output_array = output.detach().numpy()

        # Get the predicted notes by finding the index of the maximum value in each output vector
        predicted_notes = np.argmax(output_array, axis=-1)

        predicted_notes_list = [note for sublist in predicted_notes for note in sublist]  # Flattening the list

        # Calculate accuracy
        accuracy = np.mean(np.array(predicted_notes_list) == np.array(target))
        print(f'Accuracy for song {i + 1}, batch {j}: {accuracy:.2f}')
        print(f'target notes:    {target}')
        print(f'predicted notes: {predicted_notes_list}')


Accuracy for song 1, batch 0: 0.95
target notes:    [5, 0, 2, 7, 5, 3, 5, 4, 4, 3, 7, 4, 6, 1, 5, 7, 0, 2, 6, 0, 7, 0, 4, 1, 7, 2, 1, 1, 5, 6, 3, 7, 6, 3, 1, 1, 2, 0, 1, 7, 2, 2]
predicted notes: [3, 4, 2, 7, 5, 3, 5, 4, 4, 3, 7, 4, 6, 1, 5, 7, 0, 2, 6, 0, 7, 0, 4, 1, 7, 2, 1, 1, 5, 6, 3, 7, 6, 3, 1, 1, 2, 0, 1, 7, 2, 2]
Accuracy for song 1, batch 1: 0.98
target notes:    [0, 2, 7, 5, 3, 5, 4, 4, 3, 7, 4, 6, 1, 5, 7, 0, 2, 6, 0, 7, 0, 4, 1, 7, 2, 1, 1, 5, 6, 3, 7, 6, 3, 1, 1, 2, 0, 1, 7, 2, 2, 5]
predicted notes: [0, 1, 7, 5, 3, 5, 4, 4, 3, 7, 4, 6, 1, 5, 7, 0, 2, 6, 0, 7, 0, 4, 1, 7, 2, 1, 1, 5, 6, 3, 7, 6, 3, 1, 1, 2, 0, 1, 7, 2, 2, 5]
Accuracy for song 1, batch 2: 0.98
target notes:    [2, 7, 5, 3, 5, 4, 4, 3, 7, 4, 6, 1, 5, 7, 0, 2, 6, 0, 7, 0, 4, 1, 7, 2, 1, 1, 5, 6, 3, 7, 6, 3, 1, 1, 2, 0, 1, 7, 2, 2, 5, 0]
predicted notes: [1, 7, 5, 3, 5, 4, 4, 3, 7, 4, 6, 1, 5, 7, 0, 2, 6, 0, 7, 0, 4, 1, 7, 2, 1, 1, 5, 6, 3, 7, 6, 3, 1, 1, 2, 0, 1, 7, 2, 2, 5, 0]
Accuracy for song 1, batch 3: 0